# Natural Language Processing



Será usada Rede Neurais Recorrentes, aos quais são melhores para processar textos e carácteres. Faremos:


1.   Geração de carácteres
2.   Identificação de sentimento



## Sequenciando os dados
Diferente de imagens, Videos e textos precisam passar por um processo de "adaptação". Usaremos um dicionario para contar a frequência da palavra dentro do texto. Esse processo é chamado saco de palavras (bag of words). Para detectar sentimentos esse algoritimo não é muito eficaz já que ele não liga para a ordem das palavras.

'''I thought the movie was going to be bad, but it was actually amazing!'''

'''I thought the movie was going to be amazing, but it was actually bad!'''

Essas duas frases terão o mesmo peso em certos algoritmos



### Bag of Words

A frequência da palavra é contada e colocada em um dicionário, com o index da palavra. Esse algoritmo não se preocupa com a ordem das palavras no texto.  

In [ ]:
vocabulario = {}  # mapeia a frequencia da palavra
word_encoding = 1
def bag_of_words(texto):
  global word_encoding

  palavras = texto.lower().split(" ")  # divide todas as palavras do texto
  bag = {}  # coloca todas as palavras e frequencia nesse dicionário 

  for palavra in palavras:
    if palavra in vocabulario:
      encoding = vocabulario[palavra]  # pega a frequencia da palavra
    else:
      vocabulario[palavra] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

texto = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(texto)
print(bag)
print(vocabulario)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


Esse algoritmo não será utilizado, porque a frase perde a sequência.

In [ ]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


## Word Embedding
Esse método transforma a palavra em um vetor, mantem a ordem e frequência e também aproxima palavras com significados próximos.

## Recurrent Neural Networks (RNN's)

Esse tipo de rede neural usa o valor anterior para calcular o novo.
![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Sendo:

**h<sub>t</sub>** saida no tempo t

**x<sub>t</sub>** entrada no tempo t

**A** camada recorrente (loop)

Por exemplo, um texto com t palavras será calculado uma palavra no H0 e será passado o calculo para H1, no qual a segunda palavra é calculada em cima do calculo da primeria e assim por diante.

Esse método faz com que ao chegar no final(Ht) a primeira palavra seja perdida.

## LSTM - Long-Short Term Memory

O RNN anterior é chamado de simples. O LSTM usa um método para acessar os valores anteriores da camada que desaparecem no RNN simples ao longo do calculo.

## Análise de Sentimentos

Usando textos, o algoritmo classificará opiniões como boas, ruins ou neutras em relaçao a filmes.

## Database de Reviews de Filmes

Utilizando Keras, um dataset com 25.000 reviews do IMDB já processado e com labels será utilizado. Cada review já estava codificado com a frequência da palavra no dataset inteiro.

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_TAM = 88584

MAXLEN = 250
BATCH_SIZE = 64

(treino_X, treino_y), (teste_X, teste_y) = imdb.load_data(num_words = VOCAB_TAM)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

## Mais porcessamento

Como cama review não tem um valor fixo, vamos colocar todos com 250 itens.

Se for maior que 250 descartamos o restante.
Se for menor que 250 acrescentamos o restante com 0.

In [ ]:
treino_X = sequence.pad_sequences(treino_X, MAXLEN)
teste_X = sequence.pad_sequences(teste_X, MAXLEN)

## Modelo

Usamos um embedding de palavras como entrada, um LSTM como camada escondida e de saída usamo uma densa. O 32 é o número de dimensões do vetor. 

In [ ]:
modelo = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_TAM, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

## Treino



In [ ]:
modelo.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = modelo.fit(treino_X, treino_y, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 45s 20ms/step - loss: 0.5195 - acc: 0.7284 - val_loss: 0.2891 - val_acc: 0.8854
Epoch 2/10
625/625 [==============================] - 12s 19ms/step - loss: 0.2338 - acc: 0.9129 - val_loss: 0.2797 - val_acc: 0.8882
Epoch 3/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1708 - acc: 0.9376 - val_loss: 0.2648 - val_acc: 0.8900
Epoch 4/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1446 - acc: 0.9495 - val_loss: 0.2989 - val_acc: 0.8916
Epoch 5/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1165 - acc: 0.9619 - val_loss: 0.3553 - val_acc: 0.8802
Epoch 6/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1032 - acc: 0.9634 - val_loss: 0.3288 - val_acc: 0.8808
Epoch 7/10
625/625 [==============================] - 12s 19ms/step - loss: 0.0919 - acc: 0.9710 - val_loss: 0.3944 - val_acc: 0.8784
Epoch 8/10
625/625 [==============================] - 12s 19ms

In [ ]:
results = modelo.evaluate(teste_X, teste_y)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5369 - acc: 0.8345
[0.5369390845298767, 0.8344799876213074]


O modelo acertou 83,44% dos testes.

# Predição

Como o modelo tá codificado, para prever novos exemplos precisamos codificar o texto também.


In [ ]:
index_palavras = imdb.get_word_index() # Index das palavras provenientes do IMDB

def encode_text(texto):
  tokens = keras.preprocessing.text.text_to_word_sequence(texto)
  tokens = [index_palavras[palavra] if palavra in index_palavras else 0 for palavra in tokens] 
  # Se a palavra estiver no dicionario ela é add ao Token
  return sequence.pad_sequences([tokens], MAXLEN)[0] # Como é uma sequencia de listas, pegamos só a primeira

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1646592/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
# Decodifica os valores para texto

index_palavras_reverso = {value: key for (key, value) in index_palavras.items()}

def decode_integers(integers):
    texto = ""
    for num in integers:
      if num != 0:
        texto += index_palavras_reverso[num] + " "

    return texto[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# Prevendo

def predict(texto):
  texto_codificado = encode_text(texto)
  pred = np.zeros((1,250))
  pred[0] = texto_codificado
  resultado = modelo.predict(pred) 
  print(resultado[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.89084]
[0.08708591]


# Gerador de Peças

Mostrando uma peça de teatro ao RNN, faremos ele criar uma versão nova em cima do exemplo.

# Dataset

Usaremos Romeo e Julieta de Shakesphere, mas também podemos usar outro arquivo TXT prórpio.

In [11]:
caminho = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


### Arquivo proprio em TXT

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

## Lendo o Conteúdo

In [14]:
# Lê e decodifica para py2 compat.
texto = open(caminho, 'rb').read().decode(encoding='utf-8')
# tamanho do texto é o número de caracteres estão nele
print ('Tamanho do texto: {} caracteres'.format(len(texto)))

Tamanho do texto: 1115394 caracteres


## Codificando

Faremos a codificação tirando caracteres unicos do texto

In [16]:
vocabulario = sorted(set(texto))
# Cria um mapa para caracteres unicos por indices.
caract_para_indx = {u:i for i, u in enumerate(vocabulario)}
indx_para_caract = np.array(vocabulario)

def text_to_int(text):
  return np.array([caract_para_indx[c] for c in texto])

texto_como_int = text_to_int(texto)

In [17]:
print("Texto:", texto[:13])
print("codificado:", text_to_int(texto[:13]))

Texto: First Citizen
codificado: [18 47 56 ... 45  8  0]


In [19]:
# Converte de indeces para texto

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(indx_para_caract[ints])

print(int_to_text(texto_como_int[:13]))

First Citizen


## Criando exemplos de treino

Como o objetivo é criar uma nova peça, para o algoritmo gerar novos texto passaremos exemplos em que ele deve completar uma palavra ou frase.

Os exemplos de treino usaram uma sequência definada de caracteres que serviram de entra e de saida, mas com a saída a 1 caractere a direita. Exemplo:

*input: Hell | output: ello*




In [20]:
seq_length = 100  # tamanho da sequencia para treino
examplos_por_epoch = len(text)//(seq_length+1)

# Cria exemplos de treino / alvos
char_dataset = tf.data.Dataset.from_tensor_slices(texto_como_int)

Agora usamos o método de Batch para transforma esse dataset em batchs de qualquer tamanho

In [21]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [22]:
def split_input_target(chunk):  # para o exemplo: hello
    input_texto = chunk[:-1]  # hell
    target_texto = chunk[1:]  # ello
    return input_texto, target_texto  # hell, ello

dataset = sequences.map(split_input_target)  # Usamo o mapa para aplicar a função a todas as entradas de dados

In [24]:
for x, y in dataset.take(2):
  print("\n\nEXEMPLO\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXEMPLO

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXEMPLO

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finalmente construimos batches de treino

In [26]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocabulario)  # vocab é o número de caracteres unicos
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size é para embaralhar o dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)